<a href="https://colab.research.google.com/github/gasparjpv/dashboard-peaceriver/blob/main/notebook/Tratamento_de_dados_Peace_River.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Função para download das bases

def check_and_install(package):
    import subprocess
    import sys
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])


def extract_drive_id(url):
    import re
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if match:
        return match.group(1)
    else:
        return None


def download_file_from_google_drive(url_file_drive, file_name, path_destination=None):
    check_and_install('gdown')
    import gdown
    import os
    import re

    id = extract_drive_id(url=url_file_drive)
    print("ID = ",id)
    # Criar diretório se não existir
    if path_destination is not None:
        if not os.path.exists(path_destination):
            os.makedirs(path_destination)
    # URL compartilhada do Google Drive
    url = f'https://drive.google.com/uc?id={id}'

    if path_destination is not None:
        destination = path_destination
    else:
        destination = ''
    output = destination+'/'+file_name
    gdown.download(url, output, quiet=False)

In [ ]:
#@title Download das Bases
download_file_from_google_drive(url_file_drive='https://drive.google.com/file/d/1-XIJ2dcuGUjFHRxD0aOJ2iiAkP_ds990/view?usp=sharing', file_name='WIN_RESULTS_PRB.csv', path_destination='dataset')

ID =  1-XIJ2dcuGUjFHRxD0aOJ2iiAkP_ds990


Downloading...
From (original): https://drive.google.com/uc?id=1-XIJ2dcuGUjFHRxD0aOJ2iiAkP_ds990
From (redirected): https://drive.google.com/uc?id=1-XIJ2dcuGUjFHRxD0aOJ2iiAkP_ds990&confirm=t&uuid=501f0ea0-7be0-4f9f-ac70-a74dd596f65a
To: /content/dataset/WIN_RESULTS_PRB.csv
100%|██████████| 209M/209M [00:03<00:00, 61.8MB/s]


In [ ]:
#@title Instalar livrarias
check_and_install('fireducks')
# import fireducks.pandas as pd
import fireducks.pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
#@title Cria Dataframes
df_win_results_prb = pd.read_csv('dataset/WIN_RESULTS_PRB.csv', low_memory=False)

In [ ]:
#@title Tratando campos numericos com espaços e caracteres especiais

def remove_spaces_and_dots(df, column_names):
  for column_name in column_names:
    df[column_name] = df[column_name].str.replace(r'\s+', '', regex=True)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    #df[column_name] = df[column_name].fillna('')
    #pd.options.display.float_format = '{:.2f}'.format
  return df

df_win_results_prb['month'] = df_win_results_prb['month'].str.replace(r'\s+', '', regex=True)

df_win_results_prb['ACTIVITY_BOTTOM_DEPTH'] = df_win_results_prb['ACTIVITY_BOTTOM_DEPTH'].str.replace(r'\s+', '', regex=True)
df_win_results_prb['ACTIVITY_BOTTOM_DEPTH'] = df_win_results_prb['ACTIVITY_BOTTOM_DEPTH'].str.replace(r'\.', '', regex=True)

df_win_results_prb['ACTIVITY_TOP_DEPTH'] = df_win_results_prb['ACTIVITY_TOP_DEPTH'].str.replace(r'\s+', '', regex=True)
df_win_results_prb['ACTIVITY_TOP_DEPTH'] = df_win_results_prb['ACTIVITY_TOP_DEPTH'].str.replace(r'\.', '', regex=True)

colunas_a_tratar_1 = ['ACTIVITY_DEPTH', 'DEP_MDL', 'DEP_PQL', 'DEP_RESULT_VALUE', 'final_result_value']

df_win_results_prb = remove_spaces_and_dots(df_win_results_prb, colunas_a_tratar_1)

df_win_results_prb = df_win_results_prb.dropna(subset=['final_result_value'])


In [ ]:
#@title Seleção das colunas de interesse e tratamento de campo de data
df_filter = df_win_results_prb

columns_to_keep = ['ACTIVITY_START_DATE', 'ANALYSIS_METHOD', 'ANALYTE_PRIMARY_NAME', 'BASIN_NAME','County_Name', 'MONITORING_LOCATION_NAME', 'ORGANIZATION_NAME', 'PROJECT_ID', 'RESULT_KEY', "DEP_RESULT_UNIT", "final_result_value", 'SECONDARY_TYPE', 'x', 'y']
df_filter = df_filter[columns_to_keep]

df_filter['ACTIVITY_START_DATE'] = pd.to_datetime(df_filter['ACTIVITY_START_DATE'],format='%d%b%y:%H:%M:%S')

df_filter.rename(columns=lambda x: x.lower(), inplace=True)
df_filter.loc[df_filter['basin_name'].isnull(), 'basin_name'] = 'CHARLOTTE HARBOR'
df_filter['result_key'] = df_filter['result_key'].astype('str')
df_filter['activity_start_date'] = df_filter['activity_start_date'].dt.strftime('%Y-%m-%d')

In [ ]:
#@title Criar base tipo sqlite
import pandas as pd
import sqlite3
from google.colab import files

conn = sqlite3.connect('banco_dados.db')

df_filter.to_sql('minha_tabela', conn, if_exists='replace', index=False)

# Fechar a conexão
conn.close()

# Fazer o download do arquivo .db para o seu computador
files.download('banco_dados.db')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>